In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Test: Converting a DFA to a Regular Expression

In this notebook, we test the algorithm implemented in `05-DFA-2-RegExp.ipynb`.
We manually construct a simple Deterministic Finite Automaton (DFA) and convert it into a Regular Expression.

Since the raw output of the State Elimination algorithm is often extremely verbose and redundant, we use the Term Rewriting System from `Rewrite.ipynb` to simplify the result into a human-readable form.

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, RecursiveMap, Tuple } from "recursive-set";
import {
    State,
    Char,
    DFA,
    DFAState,
    TransRelDet
} from "./01-NFA-2-DFA";
import { RegExp } from "./03-RegExp-2-NFA";
import { dfa2regexp } from "./05-DFA-2-RegExp";
import { dfa2dot, dfa2string, renderLegend } from "./FSM-2-Dot";
import { simplify, regexpToString } from "./Rewrite";

## 1. Manual DFA Construction

We define a DFA with 3 states representing the sets $\{1\}, \{2\}, \{3\}$.
* Start State: $\{1\}$
* Accepting State: $\{3\}$
* Transitions:
    * $1 \xrightarrow{a} 2$
    * $2 \xrightarrow{b} 3$
    * $3 \xrightarrow{a} 2$

This automaton recognizes the language $L = a(ba)^*b$.

In [ ]:
const s = (n: number) => new RecursiveSet(n);
const [q1, q2, q3] = [s(1), s(2), s(3)];

const delta = new RecursiveMap<Tuple<[DFAState, Char]>, DFAState>();
delta.set(new Tuple(q1, "a"), q2);
delta.set(new Tuple(q2, "b"), q3);
delta.set(new Tuple(q3, "a"), q2);

const dfa: DFA = {
    Q: new RecursiveSet(q1, q2, q3),
    Σ: new RecursiveSet("a", "b"),
    δ: delta,
    q0: q1,
    A: new RecursiveSet(q3)
};

In [ ]:
dfa2string(dfa);

In [ ]:
const dot = dfa2dot(dfa);
const viz = await instance();
display.html(viz.renderString(dot, { format: "svg" }));

In [ ]:
display.html(renderLegend(dfa))

## 2. Conversion to Regular Expression

We apply the `dfa2regexp` function. Note that the resulting expression, while mathematically correct, is typically very complex due to the generic nature of the algorithm (generating many $\emptyset$ and $\varepsilon$ terms).

In [ ]:
const r: RegExp = dfa2regexp(dfa);

console.log("Complex regular expression:");
r;

## 3. Simplification

To make the expression readable, we use our **Term Rewriting System**.

In [ ]:
const s: RegExp = simplify(r);
console.log("Simplified Expression:");
s;

## 4. Final Result

We convert the simplified AST back into a string string representation. We expect `a(ba)*b`.

In [ ]:
console.log("\nFinal regular Expression:");
regexpToString(s);